# Chapter 2: Learning Python and solving algebraic equations
Introduction to the Phys212/Biol212 class
### by Ilya Nemenman, 2016-2020

Make sure you read Chapters 1 and 2 of the *Student Guide* before this notebook, and that you read Chapters 3 and 4 in parallel with this notebook. Make sure also that you in detail study the code for the cells that I provide below, not just the output of the cells. This is essential if you want to pick up on python programming quickly.

## Some Python practice 
Let's first pre-emptively review a few elements in Python coding that many of you probably will have problems with otherwise. 

### Assignments and comparisons
Let's execute the following code:

In [1]:
a = 0
a == 1
print (a)
a = 1
print(a)
b = (a == 1)
print(b)

0
1
True


Note that the first line above assigns the value of 0 to the variable `a`. The second line does not do the assignment, but instead compares `a` to 1. Since the result of the comparison is not stored anywhere, it disappears. We can verify that `a` was not changed by the comparison by simply printing it. The next line, in contrast, assigns 1 to `a`. We verify that this, in fact, happened by printing the variable. We again do the comparison of `a` to 1. Notice, however, that this time we store the result of this comparison in a variable $b$, which has the value of `True`, as we can see by printing it.

### Vector operations
You have created numpy arrays in your first lab. Why do we bother with this concept? Arrays allow us to perform vector operations -- instead of, for example, adding two arrays element by element, or copying an array element by element we can do either of these tasks in just one single operation. Let's see how this works.

In [2]:
import numpy as np #importing the numpy module

L = 10  # the size of the arrays we will create
a = np.arange(L)
b = np.ones(L)
print(a)
print(b)

c = a + b
print(c)

[0 1 2 3 4 5 6 7 8 9]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]


Notice how we didn't have to create `c` separately, and in a single addition operation we were able to add all elements of `a` and `b` into `c`. Such vector operations are, of course, more elegant than doing the same element by element. Importantly, they are also faster, as we will verify shortly. 

Note that the array `a` is an array of integers. While the arrays `b` and `c` are arrays of real (rather, floating point -- we will discuss what this means in a few lectures) numbers. Real and integer numbers are very different types of objects, and are represented very differently in a computer memory, even if sound very similar to us. Luckily, Python is very forgiving about conversion of types of variables between various integer and real types (there are many of either kind). You can add integers to reals, for example, as the line `c = a+b` above illustrates. For this, Python will automatically convert the array of integers into reals, and then add the real numbers. There are a few cases, however, where one has to be careful, and you surely will run into them. Let's try

In [3]:
print(c[1])
print(c[1.0])

2.0


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

Note that the first line works, while the second produces an error. You can only refer to an integer position in an array. There's no element with a number 1.5, 2.7, or 1.0. 

Now let's verify that vector operations are much faster than element by element ones. To measure how long something takes, we need to load a new module `Time`.

In [ ]:
import time
dir(time)

>### Your turn 2.1
Explore what different functions in this module do.

We will be using the function `time.time()`, which return the time in seconds since the beginning of some standardized epoch as a floating point number. Notice that the function `time.time()` must be called with the parenthesis at the end. Indeed, let's try:

In [ ]:
print(time.time)
print(time.time())

>### Your turn 2.2
Explain the results of the Python output above.

Now let's see how long vector and non-vector versions of the same operations take. 

In [ ]:
L = 100000        # length of the vectors we will operate with
a = np.arange(L)  # creating these long vectors
b = np.ones(L)

# now let's add a and b as a vector  operation and see how long it takes
t = time.time()   # store the current time in the variable t
c = a+b
print ('The vector addition took', time.time()-t, 'sec.')

>### Your turn 2.3
Run the cell above multiple times. Does it always time the same duration of time to execute the same command?

Now let's do the same operation using element by element addition.

In [ ]:
t = time.time()  # store the current time in the variable t

c = np.zeros(L)  # preallocate memory for an empty array
for i in np.arange(L):  # do element-by-element addition
    c[i] = a[i] + b[i]
    
print ('Element by element addition with pre-allocated memory took', time.time() - t, 'sec.')

Notice how much longer it took to execute the same operation. But this is not the worst. Let's try the following.

In [ ]:
t = time.time()  # store the current time in the variable t

c = np.arange(0) # create a null array (array with no elements)
print(c)

for i in np.arange(L): # do element-by-element addition while growing the array
    c = np.hstack((c, a[i] + b[i]))
    
print ('Element by element addition with pre-allocated memory took', time.time() - t, 'sec.')

>### Your turn 2.4
Why do we use double perentheses ((...)) in the call to `np.hstack()`?

Why does this happen? The vector operations are the fastest -- they are encoded using low-level computer language, and are optimized internally. The element-wise edition takes longer because at every point in the cycle the Python interpreter has to do a variety of checks, such as is `a` an array? Does it have an element `i`? And the same for `b` and `c`, and all of this takes time. Finally, in the third version, every time we stack the arrays together, a new array gets created. **We will discuss this in class in more detail, and I wil add a figure later to explain this.**

## Solving a quadratic equation

Let's say we want to solve a quadratic equation $$ax^2+bx+c=0.$$ The two solutions of this are given by $$x_{1,2}=\frac{-b\pm \sqrt{b^2-4ac}}{2a},$$
where $D=b^2-4ac$ is called the discriminant. Let's write a short script to solve this. 

We start with the initialization block. Normally we would import `numpy` and other modules here as well, but now we do not need to do this, this numpy was just imported above.

In [ ]:
a = 1.0  #initializing parameter values
b = 2.0
c = -2.0

Now we calculate the square root of the determinant using the `numpy.np` function, and print the two roots.

In [ ]:
sD = np.sqrt(b**2-4*a*c)
x1 = (-b+sD)/(2*a)
x2 = (-b-sD)/(2*a)

print(x1)
print(x2)

This code would work well often, but not always. For example, change `c` to 2.0 above and rerun the cell. What happened? This introduces the idea of checking of exceptions: what if the discriminant is zero? or negative? or if `a` is equal to 0? 

>### Your turn 2.5
Change the code for solving the quadratic equations to address all possible exceptions you can think of, so that the code runs for all values of the parameters `a`, `b`, and `c`. 

>### Track 2
Rewrite the code above to work with complex numbers.

>### Your turn 2.6
Additional exercise: Evaluate a binary log (log base 2) of a number 42 using at least three different sets of commands imported from `numpy`. Verify your result by taking 2 to the appropriate power and seeing if you get 42 back. For this, you will need to remind yourself how logarithms with different bases relate to each other.

## The first model: Solving for an equilibrium position of charges, a light version
We will now postpone further discussion of Python constructions for a bit, and will try to use what we know to solve computational problems. We will turn to our first model, and will follow the steps involved in constructing it, solving it on a computer, and verifying the solution

### Problem formulation
A charged particle free to move on a rod is put in the between two particles with an electric charge of the same sign affixed to the end of the rod. Where will the charge come to rest?

### Analysis
Analysis is always the first step in solving the problem. We start with asking hwat kind of model needs to be built for this problem. The model is definitely a continuous space model -- position of the particle is a continuous variable. The model is also deterministic -- there's not a stochastic element present. However, the next question -- whether the model is dynamic or static -- is already harder. At the first glance, this is a dynamic model, since we are talking about *motion* of a charge. On the other hand, we are being asked only about the final equilibrium position. Then maybe this is a static model after all -- we only need to know where the charge comes to rest. IN fact, one can solve the model both ways: either by writing down the equations of motion, modeling the motion, and then seeing where the charges settle, or by finding the equlibrium position directly, which is what we will do here.

In any case, whether we decide to solve this as a dynamic or as a static model, to ensure that there is a resting point for the charge, there has to be friction, so that the particle does not keep oscillating infinitely. Nothing is mentioned about this in the problem formulation -- but it seems like this is presumed. If we were to write a report about this problem, we would definitely need to mention this consideration  as a part of our discussion of the problem setup. Is friction the only thing we need? Is it guaranteed that with friction the particle will come to a rest? Same sign particles repel. And the repulsion is stronger when they come closer. So a particle between two other fixed one will be pushed away from both ends -- it seems reasonable that it will come to an equilibrium point at some time if friction robs it of energy. So the solution will exist -- it now makes sense to proceed to finding it by building the model.

### Model building
The equilibrium for the system will be given by $\sum \vec{F}=0$, so that if a particle has zero velocity at this point (which the friction will achieve), it stays there. Since the particle is moving on a rod -- in one dimension -- this is equivalent to $\sum {F}=0$. There are two Coulomb forces acting on the particle from the two end charges. Let's say the charges are at positions $p_1$ and $p_2$, and have charges $q_1$ and $q_2$ respectively. Let's suppose $p_1<x<p_2$, where $x$ is the position of the moving charge. The forces acting on the moving charge, which we call $q$ are then $\sum F= \frac{qq_1}{(x-p_1)^2}-\frac{qq_2}{(x-p_2)^2}$. Thus to solve the problem, we need to solve the following equation for $x$: $\frac{qq_1}{(x-p_1)^2}-\frac{qq_2}{(x-p_2)^2}=0$, or, cancelling $q$, $\frac{q_1}{(x-p_1)^2}-\frac{q_2}{(x-p_2)^2}=0$. Interestingly, the only variables we need for initialization are the three charge values, and the two end charge positions -- the initial position of the moving charge, and charge value itself seem to be not needed. Multiplying the equation by the two denominators (which, luckily, cannot be zero), we get: ${q_1}{(x-p_2)^2}-{q_2}{(x-p_1)^2}=0$. Simplifying the equation, we get: $$(q_1-q_2) x^2 +2(-q_1p_2+q_2p_1)x+(q_1p_2^2-q_2p_1^2)=0 $$. So it seems that solving this problem is equivalent to solving a simple quadratic equation.

>### Your turn 2.7
Explore the markdown code of the text above. Notice how we write equations using $\LaTeX$. Try to pick up some $\LaTeX$ in the course of this semester. This will be useful for your future carrers as scientists. Minimally, figure out how we use `$` signs for encapsulating equations, `$$` for haveing equations in their own display lines, symbols `^` and `_` for superscripts and subscripts, and a command `\frac{}{}` for writing fractions. To verify your understanding, write a formula for a ratio of two quadratic polynomials in $x$.

### Model implementation
As suggested earlier, to build a model, we start with large blocks describing chunks of the solution of the problem. We progressively break them apart into smaller and smaller blocks, until we are able to write the code for each one of the blocks. At the same time, as we are implementing the blocks, we collect pieces that need to be initialized in the initialization block, and need to be reported in the termination block. Here we realize that, in the previous lecture, we wrote a simple program that solves the quadratic equation $ax^2+bx+c=0$. Thus implementing the model is equivalent to taking the previously written solution for the quadratic equation. Thus our implementation will consist of just three blocks: (1) initialization, where we assign  $a=q_1-q_2$, $b=2(-q_1p_2+p_2x_1)$, and $q_1p_2^2+q_2p_1^2$; (2) solving the quadratic equation and getting its two solutions; and (3) termination, where, of the two solutions, we then output the one that falls between $p_1$ and $p_2$. 

### Model verification
To verify that the solution is correct, we should try it in cases where the solution is easy to guess or to estimate otherwise, and compare. For example, we can try $q_1=q_2$, where it's clear that the solution should be $x=(p_1+p_2)/2$. We could also choose a few conditions with different $q_1$, and we would expect that growing $q_1$ should push the equilibrium farther from the left end.

### Discussion
Having solved the problem, we would usually discuss our findings here, and reiterate various constraints and assumptions that entered the solution. However, this problem is too simple too write much here.

>### Your turn 2.8
Write a model for solving for the equilibrium position of the charge.

>### Your turn 2.9
Verify that the charge settles exactly in the middle between the two end charges when the latter are equal. Think of two more substantially different verification steps and implement them.

## The second model: Solving for an equilibrium position, harder version==

### Problem formulation	
A charged particle free to move on a rod is put in the between the first and the second of three immovable particles with an electric charge of the same sign affixed to the rod at various points. Where will the charge come to rest?
 
### Analysis
The same analysis as in the simpler version applies here: we will treatthis as a static, deterministic, continuous space model. As before, we need to assume that there's friction, so that the solution exists, and the moving particle comes to rest.
 
### Model building

Again, this is similar to the *light* version. As above, the equilibrium for the system will be given by $\sum {F}=0$. There are now three Coulomb forces acting on the particle from the fixed charges. Let's say the charges are at positions $p_1<p_2<p_3$, with charges $q_1$, $q_2$, and $q_3$, respectively. Let's suppose again that the movable charge is positioned between the first two $p_1<x<p_2$. Then the solution of the problem will be given by $$\frac{q_1}{(x-p_1)^2}-\frac{q_2}{(x-p_2)^2}-\frac{q_3}{(x-p_3)^2}=0.$$ We notice again that the values of the three charges and their fixed positions are the only things that are needed for the solution. We transform the last equation to $${q_1}{(x-p_2)^2}(x-x_3)^2-{q_2}{(x-p_1)^2}(x-p_3)^2 -{q_2}{(x-p_2)^2}(x-p_1)^2=0.$$ This is not a quartic equation in $x$ anymore, and solving this problem would be equivalent to finding the root of this quartic equation, which falls between $p_1$ and $p_2$. This is going to be more interesting!
 	

### Model implementation (Newton-Raphson algorithm)
The analysis above suggests that the entire model consists of solving for roots of a fourth order polynomial. How do we do this? We now have progressed to learning the first new *computational algorithm*, namely the Newton-Raphson method for finding a root $x^*$ of an algebraic equation $F(x)=0$. It allows to find a root (zero crossings) of more-or-less arbitrary functions, not just of polynomials. Note that many of the algorithms we will deal with will carry the name of Newton, often in combination with others. It is because many of the algorithms use calculus -- they solve the problem by means of analyzing various differential properties of the involved functions. Newton was one of the two main inventors of differential and integral calculus. In addition to his contribution to physics and mathematics, we can also view him as a founder of algorithmic computational sciences -- he turned his newfound calculus knowledge into algorithms, which, in the absence of computers, one could execute using paper and pencil, or rather paper and quill in the case of newton. It is not surprising that many of these algorithms ended up carrying his name. It also reminds us all how important it is to work in a new field, where a lot of discoveries are yet to be made.

>### Your turn 2.10
Review concepts from single variable differential and integral calculus. Specifically, make sure you remind yourself what Taylor expansions are.

Let us suppose that we have a guess $x_0$ for one root of a function $F(x)$. We evaluate the function, $F_0=F(x_0)$, and it turns out that the guess is not quite right, $F_0\neq 0$. We can then write the Taylor expansion for the function $F$ for the values of $x$ around $x_0$. Namely, $$F(x)\approx F_0 + G_0(x-x_0),$$ where $G_0$ is the value of the derivative of $F$ evaluated at $x_0$, $G_0=\left(dF/dx\right)_{x_0}$. But then we can get a better estimate of where the root is by solving the equation $F_0+G_0(x-x_0)=0$. This would give $x_1=x_0-F_0/G_0$ for the next estimate. Iterating this, one will get $$x_{i+1}=x_i -F_i/G_i.$$ This way we might hope that we will approach the solution progressively better and better, as we iterate more $\lim_{i\to\infty} x_i=x^*$. At some point we will need to stop the iteration. Usually, the stopping condition is a parameter that one adds to the initialization in the experiment: the tolerance $\epsilon$ for finding the solution. That is, one interrupts the recursion when $\left|x_{i+1}-x_i\right|<\epsilon$. Note that one can also choose to interrupt the solution choice when the function itself is approximately zero, and both approaches are valid. They are, in fact, equivalent for typical functions, but may give different results for various special cases -- and the discussion of this in the *Numerical recipes* book is illuminating. 

The algorithm seems to be rather straightforward, but it's not as simple as it looks. A few notes are in order. First, the algorithm will only find a root if it is initialized with the initial guess in the root's vicinity. So if one wants to find a root, one needs to have good guesses. And to find all roots, one needs to have initial guesses near all of the roots, and then run the algorithm for each of the guesses. Even then the solution may not exist -- there is no guarantee that the the iterations converge; they can enter cycles or diverge instead. We will explore some of these cases later on graphically. Most of such problems can be tracked to the fact that the iteration step involves dividing by $G(x)$, derivative of the function $F(x)$. When this derivative becomes small, changes to $x$ will be large, which contradicts the assumption we made that the algorithm is being initiated in the vicinity of the root. All sorts of problems will emerge then. A more detailed description of the method and all of these caveats can be found in the __[Numerical Recipes](http://www.aip.de/groups/soe/local/numres/bookcpdf/c9-4.pdf)__ book.

Once the Newton solver for roots has been implemented, the rest of the model design is simple. We start with initializing variables, verifying various possible exceptions, finding the root of the equation *force equal to zero*, and then outputting the root. 

>### Your work 2.11
Write down the functions $F$ and $G$ for the problem of finding an equilibrium position of the charge.

#### Solving for the equilibrium position of the charge using the Newton Raphson solver

In [ ]:
## Initialization

import numpy as np #importing the numpy module

p1 = 0 # position of three fixed charges
p2 = 10 
p3 = 12
q1 = 1 # values of three charges
q2 = 2
q3 = 0.5
x0 = 7 # initial condition
Tolerance = 1e-5 # tolerance for change before termination

## Main loop

x = x0 #initial value for starting the loop
dx = 1.0 # fake "last change in x"
while(np.abs(dx) > Tolerance):
    F = q1*(x-p2)**2*(x-p3)**2 - q2*(x-p1)**2*(x-p3)**2 - q2*(x-p2)**2*(x-p1)**2
    G = 2*q1*(x-p2)*(x-p3)**2 + 2*q1*(x-p2)**2*(x-p3) - 2*q2*(x-p1)*(x-p3)**2  -\
        2*q2*(x-p1)**2*(x-p3) - 2*q3*(x-p2)*(x-p1)**2 - 2*q3*(x-p2)**2*(x-p1)
    dx = -F/G
    x = x+dx

## Termination

print("The charge will settle at ", x, ".", sep="")

>### Your work 2.12
Modify the provided skeletal code implementing the Newton-Raphson method, try to find **all** roots of the following functions $F(x)= {\rm atan} 3x$ and $F(x)=\frac{1}{3+x^2}-\frac{1}{6}$. Do you need to modify the code to deal with convergence of the iterations for some conditions? If so, do the modification. Can you understand why some of your attempts converged to a solution, and some did not? 

**remainder of the notebook needs to be further edited**

### Model verification
To verify the model, we need to check that it produces results that we know to be correct for various special cases. As a general rule, every term, every function in the model must be verified. For this, you set all of the parameters in turn to values that make the problem easy, and compare the output to what you should have expected. In this particular case, the following verifications would be reasonable:
:*Set the initial condition outside of the leftmost or the rightmost charge. Then there should be no equilibrium, and the charge will move to infinity -- which will mean that the Newton solver won't converge. Do we observe this?
:*If we have three fixed charges, but one of them is zero, then the problem is equivalent to having two fixed charges only. And that problem we already solved previously. So we can set, in turn, the values of each of the three fixed charges to zero, and compare that the output of the program is the same as that of the two-charges quadratic solver program. An even simpler first step could be setting one of the charges to zero, and the two other equal to each other. Then the movable charge should settle exactly in the middle between the two non-zero charges. Does this?

Notice that with these checks, we verify the effects of the initial condition and the value and the position of the fixed charges to be what we expect them to be. Thus the verification is more or less complete. In addition, none of the verification steps is useless -- if you only checked the initial condition, then the effect of the fixed charges would not be explored, and so on.
 
### Discussion
Here we implemented a solution for the equilibrium position of the charge in the background of three fixed charges.  The program, based on the Newton-Raphson algorithm, found the solution. Exceptions for position of the charges and the charge values were encoded. The program output has been verified against solution of a simpler problem with only two fixed charges, and the results (hopefully) matched. We assumed here that only the equilibrium position mattered. In principle, we could have written the problem as a dynamical model, and used the Newton's laws to find the position of the charge as a function of time. The value of the position at large time would be the equilibrium position, provides there is some friction in the problem that would force the charge to stop eventually. Another simplification we assumed was that the charges are positioned in just one dimension (on a rod). For a 2-d or a 3-d configuration of the charges, we would need to extend the Newton-Raphson algorithm to be useable for multi-dimensional root finding, which goes slightly outside of the scope of this class (would require knowing multivariate calculus).